In [ ]:
from scipy.ndimage import imread
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from math import pi as π
import os
from scipy import ndimage
from numpy.linalg import det, inv
from svmutil import *

In [ ]:
img = imread("negatives/n11.png")

In [ ]:
def read_images(path):
    return [imread(path + name) for name in sorted(os.listdir(path)) if name.endswith(".png")]

def show_images(images):
    def subplot(grid, image, index, cmap=None):
        subplot = plt.subplot(grid[i*4+index])
        subplot.imshow(img, cmap=cmap)
        subplot.axis('off')
    
    grid = gridspec.GridSpec(len(images), 4, hspace=0., wspace=0., left=0, top=1, bottom=0, right=1)
    plt.figure(figsize=(100, 100))
    for i in range(len(images)):
        subplot(grid, images[i], 0)
        subplot(grid, images[i][:,:,0], 1, 'gray')
        subplot(grid, images[i][:,:,1], 2, 'gray')
        subplot(grid, images[i][:,:,2], 3, 'gray')
    grid.update(hspace=0.)
    plt.show()

def greyscale(image):
    return np.apply_along_axis(lambda x: sum(x) / 3, 2, image)
    
def rgb_features(images):
    return np.array([np.average(np.average(image, axis=0), axis=0) for image in images])

def histogram_feature(images):
    return np.array([ndimage.histogram(image, 0, 255, 5) for image in images])

def deviation_feature(images):
    return np.array([[ndimage.standard_deviation(image)] for image in images])

def laplace_feature(images):
    return np.array([[ndimage.minimum(ndimage.sobel(greyscale(image), mode='reflect'))] for image in images])

def features(images):
    f = np.concatenate((rgb_features(images), deviation_feature(images), laplace_feature(images)), axis=1)
    return f

In [ ]:
negatives = np.array(read_images("../03/negatives/"))
positives = np.array(read_images("../03/positives/"))

In [ ]:
y, x = svm_read_problem('heart_scale')
m = svm_train(y[:200], x[:200], '-c 4')
p_label, p_acc, p_val = svm_predict(y[200:], x[200:], m)

In [ ]:
x = [list(l) for l in np.concatenate((features(positives), features(negatives)), axis=0)]
x_train = x[::2]
x_test = x[1::2]
y = [1] * len(positives) + [-1] * len(negatives)
y_train = y[::2]
y_test = y[1::2]


prob = svm_problem(y_train, x_train)
for t, t_name in enumerate(("linear", "polynomial", "radial-basis", "sigmoid")):
    param = svm_parameter('-t %d -c 4 -b 1' % t)
    m = svm_train(prob, param)
    print(t_name)
    p_label, p_acc, p_val = svm_predict(y_test, x_test, m)
    print()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
